In [1]:
import test as t
from importlib import reload
from rl.experiments.common.classes import TestProgram

reload(t)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<module 'test' from '/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/src/rl/experiments/_1_same_bigger_data_fundamental/test.py'>

In [2]:
t.ignore_warnings()
_program = t
_program.init_dataset(t.dataset_name)
_program_fields = t.attr.asdict(_program)
del _program_fields["dataset"]
test_program = TestProgram(**_program_fields, algos=t.get_learned_algos(_program), metric="Annual return")
algos = t.get_learned_algos(test_program)

In [3]:
t.predicate(test_program)

AttributeError: 'TestProgram' object has no attribute 'dataset'

In [4]:
#
t.calc_performance_statistics(test_program)

#
start_end = t.get_start_end(test_program.dataset)
print(f"Start: {start_end['start']}  -  End: {start_end['end']}")

NameError: name 'metric' is not defined

In [22]:
#
baseline = t.Baseline(ticker="SPY", start=start_end["start"], end=start_end["end"])

# compare = t.CompareAlgoBaseline(algos=sorted_list, baseline=baseline)
# compare.plot_baseline()

KeyError: 'account_value'